In [103]:
import pandas as pd
import numpy as np

**load data**

In [104]:
data_path = '/Volumes/HD/2019/out/rb-1min/'
files = os.listdir(data_path)
data_list = [pd.read_csv(f"{data_path}/{file}", index_col='datetime') for file in files]
len(data_list)

3108

In [105]:
df = pd.concat(data_list).sort_index()
df

,open,high,low,close,symbol,prev_settle,prev_close,prev_open_interest,day_open,day_high,...,open_ask_price1,open_ask_volume1,close_ask_price1,close_ask_volume1,high_bid_price1,low_bid_price1,high_ask_price1,low_ask_price1,open_vwp,close_vwp
datetime,,,,,,,,,,,,,,,,,,,,,
2019-01-02 09:00:00,3543.0,3543.0,3543.0,3543.0,rb1902,3554.0,3543.0,9220.0,0.0,0.0,...,3598.0,1.0,3594.0,18.0,3522.0,3520.0,3598.0,3594.0,3546.000000,3586.800000
2019-01-02 09:00:00,3399.0,3405.0,3389.0,3401.0,rb1905,3400.0,3404.0,2374342.0,3398.0,3411.0,...,3400.0,291.0,3401.0,61.0,3404.0,3388.0,3406.0,3389.0,3399.299691,3400.089574
2019-01-02 09:00:00,3269.0,3269.0,3256.0,3267.0,rb1909,3268.0,3269.0,4982.0,0.0,3267.0,...,3315.0,1.0,3269.0,1.0,3268.0,3208.0,3315.0,3256.0,3261.500000,3265.500000
2019-01-02 09:00:00,3138.0,3138.0,3123.0,3123.0,rb1912,3133.0,3138.0,766.0,0.0,3123.0,...,3188.0,3.0,3151.0,2.0,3132.0,3097.0,3188.0,3142.0,3155.500000,3112.600000
2019-01-02 09:00:00,3352.0,3356.0,3352.0,3356.0,rb1906,3349.0,3352.0,2052.0,0.0,3356.0,...,3432.0,3.0,3352.0,3.0,3347.0,3268.0,3432.0,3344.0,3350.000000,3350.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-29 14:59:00,3618.0,3621.0,3617.0,3620.0,rb2001,3587.0,3618.0,2333174.0,3616.0,3644.0,...,3619.0,333.0,3620.0,79.0,3620.0,3617.0,3621.0,3618.0,3618.154381,3619.341991
2019-11-29 14:59:00,3427.0,3429.0,3426.0,3427.0,rb2005,3386.0,3418.0,1622948.0,3420.0,3451.0,...,3428.0,638.0,3427.0,135.0,3429.0,3426.0,3430.0,3427.0,3427.672287,3426.336658
2019-11-29 14:59:00,3505.0,3505.0,3505.0,3505.0,rb2003,3460.0,3486.0,14038.0,3485.0,3525.0,...,3510.0,5.0,3510.0,5.0,3505.0,3491.0,3510.0,3510.0,3508.285714,3506.000000


**Train develop set split**

In [106]:
tickers = sorted(set(df.symbol))
num_tickers = len(tickers)
print(f"num_tickers: {num_tickers}")
train_dev_split = int(0.75*num_tickers)
train_tickers = set(tickers[:train_dev_split])
dev_tickers = set(tickers[train_dev_split:])
print(f"train_tickers: {train_tickers}")
print(f"dev_tickers: {dev_tickers}")

train = df[df.symbol.map(lambda x: x in train_tickers)]
dev = df[df.symbol.map(lambda x: x in dev_tickers)]
print(f"train: {train.shape}, dev: {dev.shape}")

num_tickers: 23
train_tickers: {'rb1905', 'rb2004', 'rb2005', 'rb1906', 'rb1903', 'rb1901', 'rb1910', 'rb1911', 'rb1912', 'rb2001', 'rb2002', 'rb1907', 'rb1904', 'rb1909', 'rb2003', 'rb1902', 'rb1908'}
dev_tickers: {'rb2006', 'rb2011', 'rb2010', 'rb2007', 'rb2009', 'rb2008'}
train: (857166, 30), dev: (116717, 30)


In [107]:
def calc(train, train_tickers):
    for symbol in train_tickers:
        index = train.symbol == symbol
        train.loc[index, 'next_rtn'] = train.loc[index, 'close'].shift(-1) / train.loc[index, 'close'] - 1
        train.loc[index, 'next_vw_rtn'] = train.loc[index, 'close_vwp'].shift(-1) / train.loc[index, 'close_vwp'] - 1
        train.loc[index, 'rtn'] = train.loc[index, 'close'].pct_change() # percentage change
        train.loc[index, 'next_log_rtn'] = np.log(train.loc[index, 'close'].shift(-1)) - np.log(train.loc[index, 'close'])

        train.loc[index, 'bar_volume'] = train.loc[index, 'volume'] - train.loc[index, 'volume'].shift().fillna(0)
        train.loc[index & train.bar_volume < 0, 'bar_volume'] = train.loc[index & train.bar_volume < 0, 'volume']
        train.loc[index, 'bar_volume'] = train.loc[index, 'bar_volume'] / 2

        train.loc[index, 'bar_turnover'] = train.loc[index, 'turnover'] - train.loc[index, 'turnover'].shift().fillna(0)
        train.loc[index & train.bar_turnover < 0, 'bar_turnover'] = train.loc[index & train.bar_turnover < 0, 'turnover']

        train.loc[index, 'prev_bar_volume'] = train.loc[index, 'bar_volume'].shift()
        train.loc[index, 'mean_rtn_of_prev_5_bar'] = train.loc[index, 'rtn'].rolling(5, win_type=None).mean()
        train.loc[index, 'mean_rtn_of_prev_20_bar'] = train.loc[index, 'rtn'].rolling(20, win_type=None).mean()
        train.loc[index, 'mean_rtn_of_prev_60_bar'] = train.loc[index, 'rtn'].rolling(60, win_type=None).mean()

        train.loc[index, 'mean_volume_of_prev_5_bar'] = train.loc[index, 'bar_volume'].rolling(5, win_type=None).mean()
        train.loc[index, 'mean_volume_of_prev_20_bar'] = train.loc[index, 'bar_volume'].rolling(20, win_type=None).mean()

        train.loc[index, 'mean_openinterest_of_prev_5_bar'] = train.loc[index, 'open_interest'].rolling(5, win_type=None).mean()
        train.loc[index, 'mean_openinterest_of_prev_20_bar'] = train.loc[index, 'open_interest'].rolling(20, win_type=None).mean()

        train.loc[index, 'prev_bar_turnover'] = train.loc[index, 'bar_turnover'].shift()

        train.loc[index, 'mean_turnover_of_prev_5_bar'] = train.loc[index, 'bar_turnover'].rolling(5, win_type=None).mean()
        train.loc[index, 'mean_turnover_of_prev_20_bar'] = train.loc[index, 'bar_turnover'].rolling(20, win_type=None).mean()
        train.loc[index, 'prev_rtn'] = train.loc[index, 'rtn'].shift()
    
calc(train, train_tickers)
calc(dev, dev_tickers)

In [108]:
result = train[['next_rtn', 'next_vw_rtn', 'rtn', 'next_log_rtn', 'bar_volume', 'bar_turnover', 'prev_bar_volume', 'mean_rtn_of_prev_5_bar', 'mean_rtn_of_prev_20_bar', 'mean_rtn_of_prev_60_bar', 'mean_volume_of_prev_5_bar', 'mean_volume_of_prev_20_bar', 'mean_openinterest_of_prev_5_bar', 'mean_openinterest_of_prev_20_bar', 'prev_bar_turnover', 'mean_turnover_of_prev_5_bar', 'mean_turnover_of_prev_20_bar', 'prev_rtn']]

dev_result = dev[['next_rtn', 'next_vw_rtn', 'rtn', 'next_log_rtn', 'bar_volume', 'bar_turnover', 'prev_bar_volume', 'mean_rtn_of_prev_5_bar', 'mean_rtn_of_prev_20_bar', 'mean_rtn_of_prev_60_bar', 'mean_volume_of_prev_5_bar', 'mean_volume_of_prev_20_bar', 'mean_openinterest_of_prev_5_bar', 'mean_openinterest_of_prev_20_bar', 'prev_bar_turnover', 'mean_turnover_of_prev_5_bar', 'mean_turnover_of_prev_20_bar', 'prev_rtn']]

result[['rtn', 'next_rtn']].corr()

,rtn,next_rtn
rtn,1.000000,-0.062776
next_rtn,-0.062776,1.000000


In [109]:
result.columns[4:]

Index(['bar_volume', 'bar_turnover', 'prev_bar_volume',
       'mean_rtn_of_prev_5_bar', 'mean_rtn_of_prev_20_bar',
       'mean_rtn_of_prev_60_bar', 'mean_volume_of_prev_5_bar',
       'mean_volume_of_prev_20_bar', 'mean_openinterest_of_prev_5_bar',
       'mean_openinterest_of_prev_20_bar', 'prev_bar_turnover',
       'mean_turnover_of_prev_5_bar', 'mean_turnover_of_prev_20_bar',
       'prev_rtn'],
      dtype='object')

In [110]:
for col in result.columns[4:]:
    data = result.dropna()
    x = data[col]
    y = data.next_rtn
    corr = np.corrcoef(x,y)[0,1]
    print(col, corr)

bar_volume 2.546408722643824e-05
bar_turnover 2.436583650243473e-05
prev_bar_volume 0.0015575065776185686
mean_rtn_of_prev_5_bar -0.06490281853119778
mean_rtn_of_prev_20_bar -0.05115782323978411
mean_rtn_of_prev_60_bar -0.0323284899243997
mean_volume_of_prev_5_bar 0.0008444879244011606
mean_volume_of_prev_20_bar 0.0007972757278593856
mean_openinterest_of_prev_5_bar -0.0008733886354782134
mean_openinterest_of_prev_20_bar -0.0008711816675252179
prev_bar_turnover 0.001541834210625902
mean_turnover_of_prev_5_bar 0.0007947507433577122
mean_turnover_of_prev_20_bar 0.0007385269256789401
prev_rtn -0.032907674697611244


In [112]:
from sklearn.linear_model import LinearRegression
data = result.dropna()
x = data.loc[:, data.columns[4:]]
y1 = data.next_rtn
y2 = data.next_vw_rtn
reg1 = LinearRegression().fit(x,y1)
print(reg1.score(x,y1))
print(reg1.coef_)

reg2 = LinearRegression().fit(x,y2)
print(reg2.score(x,y2))
print(reg2.coef_)



0.004815540876648039
[-2.52031542e-10  3.40156710e-15 -1.47306648e-10 -1.19983464e-01
 -1.29795963e-01 -3.60948145e-02  6.05144255e-10  1.71302469e-09
 -8.42796159e-10  8.42013521e-10  2.49745440e-15 -8.04128401e-15
 -2.27064642e-14 -7.84637298e-03]
4.739606688397568e-05
[-2.60927913e-10  3.55532239e-15 -4.13919674e-11 -3.05996493e-02
 -1.11431276e-02  3.82909631e-02  8.58810842e-10  2.20747333e-09
 -4.80048664e-10  4.78869896e-10  9.85749042e-16 -1.17645336e-14
 -2.89154532e-14 -2.21705894e-03]


In [119]:
data_dev = dev_result.dropna()
x_dev = data_dev.loc[:, dev_result.columns[4:]]
y1_dev = data_dev.next_rtn
y2_dev = data_dev.next_vw_rtn
print(reg1.score(x_dev, y1_dev))
print(reg2.score(x_dev, y2_dev))

0.0040515363217588085
-2.9211626295655432e-05
